In [1]:
!pip install --upgrade tensorflow-gpu==1.13.0rc1

     |████████████████████████████████| 345.2MB 60kB/s 
     |████████████████████████████████| 368kB 38.7MB/s 
     |████████████████████████████████| 3.1MB 20.4MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.12.2 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0


In [2]:
import tensorflow as tf

tf.__version__

'1.13.0-rc1'

In [0]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

In [6]:
!rm -rf data_in;rm -rf data_out;ls;

adc.json  gdrive  sample_data


In [0]:
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)
    
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

In [8]:
!wget -P '/content/gdrive/My Drive/Colab Notebooks/Data/' https://raw.githubusercontent.com/changwookjun/learningspoons/master/Data/nsmc_train_input.npy;
!wget -P '/content/gdrive/My Drive/Colab Notebooks/Data/' https://raw.githubusercontent.com/changwookjun/learningspoons/master/Data/nsmc_train_label.npy;
!wget -P '/content/gdrive/My Drive/Colab Notebooks/Data/' https://raw.githubusercontent.com/changwookjun/learningspoons/master/Data/data_configs.json;

--2019-07-05 22:00:41--  https://raw.githubusercontent.com/changwookjun/learningspoons/master/Data/nsmc_train_input.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4800128 (4.6M) [application/octet-stream]
Saving to: ‘/content/gdrive/My Drive/Colab Notebooks/Data/nsmc_train_input.npy.3’

nsmc_train_input.np 100%[===================>]   4.58M  --.-KB/s    in 0.06s   

2019-07-05 22:00:41 (72.5 MB/s) - ‘/content/gdrive/My Drive/Colab Notebooks/Data/nsmc_train_input.npy.3’ saved [4800128/4800128]

--2019-07-05 22:00:42--  https://raw.githubusercontent.com/changwookjun/learningspoons/master/Data/nsmc_train_label.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubuserco

In [0]:
!cp '/content/gdrive/My Drive/Colab Notebooks/Data/nsmc_train_input.npy' ./data_in/nsmc_train_input.npy;
!cp '/content/gdrive/My Drive/Colab Notebooks/Data/nsmc_train_label.npy' ./data_in/nsmc_train_label.npy;
!cp '/content/gdrive/My Drive/Colab Notebooks/Data/data_configs.json' ./data_in/data_configs.json;

In [10]:
!ls -al ./data_in/

total 6684
drwxr-xr-x 2 root root    4096 Jul  5 22:00 .
drwxr-xr-x 1 root root    4096 Jul  5 22:00 ..
-rw-r--r-- 1 root root  830764 Jul  5 22:00 data_configs.json
-rw-r--r-- 1 root root 4800128 Jul  5 22:00 nsmc_train_input.npy
-rw-r--r-- 1 root root 1200128 Jul  5 22:00 nsmc_train_label.npy


In [0]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

input_data = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA, 'rb'))
label_data = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA, 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [0]:
TEST_SPLIT = 0.1
RNG_SEED = 13371447
VOCAB_SIZE = prepro_configs['vocab_size']
EMB_SIZE = 128
BATCH_SIZE = 64 
NUM_EPOCHS = 1
HIDDEN_SIZE = 128
MAX_SEQUENCE_LENGTH = 8
SCALE_OUT = 2
ATTENITON_SIZE = 128 * SCALE_OUT
ATTENITON_HOP_SIZE = ATTENITON_SIZE 

input_train, input_eval, label_train, label_eval = train_test_split(input_data, label_data, test_size=TEST_SPLIT, random_state=RNG_SEED)

In [0]:
def mapping_fn(X, Y):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=len(input_train))
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
    dataset = dataset.shuffle(buffer_size=len(input_eval))
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [0]:
def model_fn(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    initializer = tf.contrib.layers.xavier_initializer()
    
    
    embedding_layer = tf.keras.layers.Embedding(
                            VOCAB_SIZE,
                            EMB_SIZE)(features['x'])
    
    dropout_emb = tf.keras.layers.Dropout(rate = 0.5)(embedding_layer) # ? , 8, 128

    ### Model start
    lstm_cell_fw = tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) 
    
    lstm_cell_bw = tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) 
    
    ((fw_outputs, bw_outputs), states) = tf.nn.bidirectional_dynamic_rnn(cell_fw=lstm_cell_fw, # ?, 8, 128
                                                                             cell_bw=lstm_cell_bw, # ?, 8, 128
                                                                             inputs=dropout_emb,
                                                                             dtype=tf.float32)
    
    hidden = tf.concat([fw_outputs, bw_outputs], axis=2) #?, 8, 256
    
    a_w1 = tf.get_variable("a_w1", shape=[BATCH_SIZE, HIDDEN_SIZE * 2, ATTENITON_HOP_SIZE], initializer=initializer) # ?,  256, 256
    
    result1 = tf.nn.tanh(tf.matmul(hidden, a_w1)) # ? 8, 256
    
    a_w2 = tf.get_variable("a_w2", shape=[BATCH_SIZE, ATTENITON_HOP_SIZE, ATTENITON_SIZE], initializer=initializer) # ?, 256, 256
    
    result2 = tf.matmul(result1, a_w2) # ? 8, 256
    
    result2_reshape = tf.transpose(result2, [0, 2, 1]) # ?, 256, 8
      
    score = tf.nn.softmax(result2_reshape) # ?, 256, 8
    
    score = tf.transpose(score, [0, 2, 1]) # ?, 8, 256
    
    last_metrix = hidden * score # ?, 8, 256
    
    last_metrix = tf.reshape(last_metrix, [-1, MAX_SEQUENCE_LENGTH * ATTENITON_HOP_SIZE]) # ?, 2048

    logits = tf.keras.layers.Dense(units=1)(last_metrix) # ?, 1
    
    if labels is not None:
        labels_ = tf.reshape(labels, [-1, 1]) # ?, 1
    
    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels_, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels_, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        )

In [15]:
!rm -rf data_out;ls;ls -al ./data_in

adc.json  data_in  gdrive  sample_data
total 6684
drwxr-xr-x 2 root root    4096 Jul  5 22:00 .
drwxr-xr-x 1 root root    4096 Jul  5 22:00 ..
-rw-r--r-- 1 root root  830764 Jul  5 22:00 data_configs.json
-rw-r--r-- 1 root root 4800128 Jul  5 22:00 nsmc_train_input.npy
-rw-r--r-- 1 root root 1200128 Jul  5 22:00 nsmc_train_label.npy


In [16]:
est = tf.estimator.Estimator(model_fn, model_dir="data_out/checkpoint/self_attention")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'data_out/checkpoint/self_attention', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faf83cb1c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
est.train(train_input_fn)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `

In [18]:
valid = est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-05T22:06:49Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from data_out/checkpoint/self_attention/model.ckpt-2109
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-05-22:06:55
INFO:tensorflow:Saving dict for global step 2109: acc = 0.82558763, global_step = 2109, loss = 0.3829688
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2109: data_out/checkpoint/self_attention/model.ckpt-2109
